In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/water-consumption/Household water consumption dataset/Data Description/Tank 2-Data Description.xlsx
/kaggle/input/water-consumption/Household water consumption dataset/Data Description/Tank 1-Data Description.xlsx
/kaggle/input/water-consumption/Household water consumption dataset/Tank 2-Data Files/3.xlsx
/kaggle/input/water-consumption/Household water consumption dataset/Tank 2-Data Files/9.xlsx
/kaggle/input/water-consumption/Household water consumption dataset/Tank 2-Data Files/4.xlsx
/kaggle/input/water-consumption/Household water consumption dataset/Tank 2-Data Files/12.xlsx
/kaggle/input/water-consumption/Household water consumption dataset/Tank 2-Data Files/14.xlsx
/kaggle/input/water-consumption/Household water consumption dataset/Tank 2-Data Files/1.xlsx
/kaggle/input/water-consumption/Household water consumption dataset/Tank 2-Data Files/7.xlsx
/kaggle/input/water-consumption/Household water consumption dataset/Tank 2-Data Files/11.xlsx
/kaggle/input/water-consu

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras import layers
from tensorflow.keras.layers import LSTM, Dense, GRU
from sklearn.metrics import r2_score, mean_squared_error


import warnings
warnings.filterwarnings('ignore')

In [3]:
folder_path = '/kaggle/input/water-consumption/Household water consumption dataset/Tank 2-Data Files'
files_list = glob.glob(folder_path + '/*.xlsx')
dfs = []
for file in files_list:
    df = pd.read_excel(file, parse_dates=(['created_at']))
    dfs.append(df)
    
merged_df = pd.concat(dfs, ignore_index=True)
merged_df.head()

,created_at,entry_id,field1
0,2018-02-02 04:59:00,17392,25.0
1,2018-02-02 04:59:19,17393,25.0
2,2018-02-02 04:59:38,17394,25.0
3,2018-02-02 04:59:58,17395,25.0
4,2018-02-02 05:00:17,17396,25.0


In [4]:
merged_df['created_at'] = pd.to_datetime(merged_df['created_at'], format='%d/%m/%Y %H:%M')
merged_df["year"] = merged_df["created_at"].dt.year
merged_df["month"] = merged_df["created_at"].dt.month
merged_df["day"] = merged_df["created_at"].dt.day
merged_df["hour"] = merged_df["created_at"].dt.hour
merged_df["minute"] = merged_df["created_at"].dt.minute
merged_df.head()

,created_at,entry_id,field1,year,month,day,hour,minute
0,2018-02-02 04:59:00,17392,25.0,2018,2,2,4,59
1,2018-02-02 04:59:19,17393,25.0,2018,2,2,4,59
2,2018-02-02 04:59:38,17394,25.0,2018,2,2,4,59
3,2018-02-02 04:59:58,17395,25.0,2018,2,2,4,59
4,2018-02-02 05:00:17,17396,25.0,2018,2,2,5,0


In [5]:
null_data = merged_df.isna().sum()
duplicate_rows = merged_df.duplicated()
print('Nulls', null_data)
print('Duplicates', duplicate_rows)

Nulls created_at    0
entry_id      0
field1        0
year          0
month         0
day           0
hour          0
minute        0
dtype: int64
Duplicates 0         False
1         False
2         False
3         False
4         False
          ...  
171692    False
171693    False
171694    False
171695    False
171696    False
Length: 171697, dtype: bool


In [6]:
# Detecting outliers using Z-score or IQR
# Let's focus on the "field1" column for outlier detection
Q1 = merged_df["field1"].quantile(0.25)
Q3 = merged_df["field1"].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR


# Identify and remove outliers
df_no_outliers = merged_df[
    (merged_df["field1"].iloc[:] >= lower_bound) &
    (merged_df["field1"].iloc[:] <= upper_bound)
]
df_no_outliers.head()

,created_at,entry_id,field1,year,month,day,hour,minute
0,2018-02-02 04:59:00,17392,25.0,2018,2,2,4,59
1,2018-02-02 04:59:19,17393,25.0,2018,2,2,4,59
2,2018-02-02 04:59:38,17394,25.0,2018,2,2,4,59
3,2018-02-02 04:59:58,17395,25.0,2018,2,2,4,59
4,2018-02-02 05:00:17,17396,25.0,2018,2,2,5,0


In [7]:
X = df_no_outliers.drop(['field1', 'created_at'], axis=1)
y = df_no_outliers['field1']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# Applying Standard Scaling to the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [9]:
# Reshape data for LSTM and  gru (samples, timesteps, features)
X_train_reshaped = X_train_scaled.reshape(X_train_scaled.shape[0], 1, X_train_scaled.shape[1])
X_test_reshaped = X_test_scaled.reshape(X_test_scaled.shape[0], 1, X_test_scaled.shape[1])

In [10]:
# lstm_model = Sequential([
#     LSTM(128, activation='relu', input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2]), return_sequences=True),
#     LSTM(64, activation='relu'),
#     Dense(1)  # Output layer
# ])

# lstm_model.compile(optimizer='adam', loss='mean_squared_error')

# # Use early stopping to prevent overfitting
# early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# lstm_model.fit(X_train_reshaped, y_train, epochs=50, validation_split=0.2, callbacks=[early_stopping])

# # Evaluate the LSTM model
# y_pred_lstm = lstm_model.predict(X_test_reshaped)

# # Calculate and print R-squared
# r_squared_lstm = r2_score(y_test, y_pred_lstm)
# print("LSTM R-squared:", r_squared_lstm)

Epoch 1/50
3347/3347 [==============================] - 19s 5ms/step - loss: 357.4482 - val_loss: 291.9199
Epoch 2/50
3347/3347 [==============================] - 15s 5ms/step - loss: 286.0229 - val_loss: 278.9136
Epoch 3/50
3347/3347 [==============================] - 17s 5ms/step - loss: 248.6742 - val_loss: 221.1562
Epoch 4/50
3347/3347 [==============================] - 16s 5ms/step - loss: 210.2983 - val_loss: 199.7903
Epoch 5/50
3347/3347 [==============================] - 17s 5ms/step - loss: 190.2350 - val_loss: 178.8681
Epoch 6/50
3347/3347 [==============================] - 16s 5ms/step - loss: 165.5320 - val_loss: 150.8909
Epoch 7/50
3347/3347 [==============================] - 17s 5ms/step - loss: 126.5430 - val_loss: 104.3700
Epoch 8/50
3347/3347 [==============================] - 15s 5ms/step - loss: 81.9238 - val_loss: 58.6855
Epoch 9/50
3347/3347 [==============================] - 16s 5ms/step - loss: 53.1956 - val_loss: 45.3628
Epoch 10/50
3347/3347 [==================

In [12]:
# r_squared = r2_score(y_test, y_pred_lstm)
# mae = mean_absolute_error(y_test, y_pred_lstm)
# rmse = mean_squared_error(y_test, y_pred_lstm, squared=False)
# mse = mean_squared_error(y_test, y_pred_lstm)

# print("TANK1:")
# print("R-squared:", r_squared)
# print("Mean Absolute Error:", mae)
# print("Root Mean Squared Error:", rmse)
# print("Mean Squared Error:", mse)

TANK1:
R-squared: 0.9840452769267113
Mean Absolute Error: 1.2630302629077226
Root Mean Squared Error: 2.354017631846011
Mean Squared Error: 5.541399011041902


In [14]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
import kerastuner as kt

# Define a function to build the LSTM model with hyperparameters
def build_model(hp):
    model = keras.Sequential()
    model.add(LSTM(units=hp.Int('units', min_value=32, max_value=256, step=32),
                   activation=hp.Choice('activation', values=['relu', 'tanh']),
                   return_sequences=True,
                   input_shape=(1, X_train_scaled.shape[1])))
    model.add(LSTM(units=hp.Int('units', min_value=32, max_value=128, step=32),
                   activation=hp.Choice('activation', values=['relu', 'tanh'])))
    model.add(Dense(1))

    # Compile the model
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='mse',
                  metrics=['mse'])
    return model

# Define the tuner and search space
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=10,  # Number of trials to run
    directory='my_tuning_directory',  # Directory to save results
    project_name='my_lstm_tuning'  # Name for this tuning project
)

# Perform hyperparameter tuning
tuner.search(X_train_reshaped, y_train, epochs=50, validation_data=(X_test_reshaped, y_test), callbacks=[early_stopping])

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the best model
best_model = tuner.hypermodel.build(best_hps)

# Train the best model
best_model.fit(X_train_reshaped, y_train, epochs=100, validation_data=(X_test_reshaped, y_test), callbacks=[early_stopping])

# Evaluate the best model
y_pred = best_model.predict(X_test_reshaped)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error (MSE) for Best Model:", mse)

Trial 10 Complete [00h 14m 57s]
val_loss: 3.524121046066284

Best val_loss So Far: 2.0956525802612305
Total elapsed time: 03h 53m 00s
Epoch 1/100
4183/4183 [==============================] - 39s 8ms/step - loss: 341.1127 - mse: 341.1127 - val_loss: 295.5520 - val_mse: 295.5520
Epoch 2/100
4183/4183 [==============================] - 33s 8ms/step - loss: 282.7931 - mse: 282.7931 - val_loss: 271.7599 - val_mse: 271.7599
Epoch 3/100
4183/4183 [==============================] - 31s 8ms/step - loss: 226.6319 - mse: 226.6319 - val_loss: 190.4625 - val_mse: 190.4625
Epoch 4/100
4183/4183 [==============================] - 32s 8ms/step - loss: 156.4347 - mse: 156.4347 - val_loss: 130.0904 - val_mse: 130.0904
Epoch 5/100
4183/4183 [==============================] - 32s 8ms/step - loss: 103.3890 - mse: 103.3890 - val_loss: 81.8784 - val_mse: 81.8784
Epoch 6/100
4183/4183 [==============================] - 31s 7ms/step - loss: 69.0744 - mse: 69.0744 - val_loss: 56.3356 - val_mse: 56.3356
Epoch 7/

In [15]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

r_squared = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
mse = mean_squared_error(y_test, y_pred)

print("TANK2:")
print("R-squared:", r_squared)
print("Mean Absolute Error:", mae)
print("Root Mean Squared Error:", rmse)
print("Mean Squared Error:", mse)

TANK2:
R-squared: 0.9928501777131379
Mean Absolute Error: 0.6301422638430128
Root Mean Squared Error: 1.5758421034625094
Mean Squared Error: 2.483278335045146
